## Imports

In [1]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 1.4 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 3.4 MB/s eta 0:00:00a 0:00:01


In [2]:
import mlflow
from mlflow import MlflowClient
from mlflow.types.schema import Schema, ColSpec
from mlflow.types import ParamSchema, ParamSpec
from mlflow.models import ModelSignature
from transformers import pipeline
import torch
import json
import os

2024-09-17 09:55:54.229364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 09:55:55.450285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Model

In [3]:
MODEL = "morgana-rodrigues/bert_qa"

In [4]:
qa = pipeline(
    'question-answering',
    model=MODEL,
    device=-1 # -1 means running on CPU
)

config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [11]:
class DistilBERTModel(mlflow.pyfunc.PythonModel):
    def _preprocess(self, inputs):
        context = inputs['context'][0]
        question = inputs['question'][0]
        print("pre processing", context,question)
        return context, question
        
    def load_context(self, context):
        self.model = pipeline(
            'question-answering',
             model=context.artifacts["model"],
             device=-1
        )
        
    def predict(self, context, model_input, params):
        in_ctx, question = self._preprocess(model_input)
        output = self.model(context=in_ctx, question=question)
        return output

    @classmethod
    def log_model(cls, model_name, trainer = None, pipeline = None, demo_folder="demo"): #eg (model, '', 'my_model')
        input_schema = Schema(
            [
                ColSpec("string", "context"),
                ColSpec("string", "question"),
            ]
        )
        output_schema = Schema(
            [
                ColSpec("string", "answer")
            ]
        )
        
        params_schema = ParamSchema(
            [
                ParamSpec("show_score", "boolean", False)
            ]
        )
      
        signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=params_schema)
        if trainer is not None:
            trainer.save_model(model_name)
        elif pipeline is not None:
            pipeline.save_pretrained(model_name)
             
        requirements = [
            "transformers==4.37.0",
            "numpy==1.24.3",
            "torch==2.0.0",
            "tqdm==4.65.0",
        ]
        mlflow.pyfunc.log_model(
            model_name,
            python_model=cls(),
            artifacts={"model": model_name, "demo": demo_folder},
            signature=signature,
            pip_requirements=requirements
        )

## Model Registry

In [12]:
mlflow.set_experiment(experiment_name='BERT for Q&A')

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 317, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/opt/conda/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 410, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/opt/conda/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1341, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/opt/conda/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1334, in _read_helper
    result = read_yaml(root, file_name)
  File "/opt/conda/lib/python3.10/site-packages/mlflow/utils/file_utils.py", line 309, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.MissingConfigException: Yaml file '/phoenix/mlflow/tmp/meta.yaml' does not 

<Experiment: artifact_location='/phoenix/mlflow/360600271393760861', creation_time=1726566996472, experiment_id='360600271393760861', last_update_time=1726566996472, lifecycle_stage='active', name='BERT for Q&A', tags={}>

In [13]:
with mlflow.start_run(run_name='BERT_QA') as run:
    print(f"Run's Artifact URI: {run.info.artifact_uri}")
    DistilBERTModel.log_model(model_name='BERT_QA', pipeline=qa)
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/BERT_QA", name='BERT_QA')

Run's Artifact URI: /phoenix/mlflow/360600271393760861/7e574bc43fb4427491d68b4a33b3b437/artifacts


2024/09/17 09:58:15 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - transformers (current: 4.44.2, required: transformers==4.37.0)
 - torch (current: 2.2.1+cu118, required: torch==2.0.0)
 - tqdm (current: 4.66.3, required: tqdm==4.65.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'BERT_QA' already exists. Creating a new version of this model...
Created version '23' of model 'BERT_QA'.


## Testing latest model registred

In [9]:
client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions("BERT_QA", stages=["None"])
latest_model_version = model_metadata[0].version
print(latest_model_version, mlflow.models.get_model_info(f"models:/BERT_QA/{latest_model_version}").signature)

2 inputs: 
  ['context': string, 'question': string]
outputs: 
  ['answer': string]
params: 
  ['show_score': boolean (default: False)]



In [10]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/BERT_QA/{latest_model_version}")
context = "Marta is mother of John and Amanda"
question = "what is the name of Marta's daugther?"
model.predict({"context": [context], "question":[question]})

2024/03/22 19:43:23 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - transformers (current: 4.39.1, required: transformers==4.37.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


pre processing ['Marta is mother of John and Amanda'] ["what is the name of Marta's daugther?"]


{'score': 0.6202739477157593, 'start': 28, 'end': 34, 'answer': 'Amanda'}